In [2]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np

# 1. Đọc dữ liệu
df = pd.read_csv("/kaggle/input/aio-2025-linear-forecasting-challenge/FPT_train.csv")   # phải có cột 'close'

# 2. Chuẩn bị dữ liệu: dùng close[t] dự báo close[t+1]
df["close_next"] = df["close"].shift(-1)
df = df.dropna()

X = df[["close"]].values          # input: close[t]
y = df["close_next"].values       # output: close[t+1]

# 3. Tách train test (80 20) để tính MSE (evaluation)
split = int(len(df) * 0.8)
X_train, X_test = X[:split], X[split:]
y_train, y_test = y[:split], y[split:]

model = LinearRegression()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print("MSE (train 80%, test 20%):", mse)

# 4. Sau đó train lại trên FULL dữ liệu trước khi dự báo 100 ngày
model_full = LinearRegression()
model_full.fit(X, y)

print("Model full data - Intercept:", model_full.intercept_)
print("Model full data - Coef:", model_full.coef_)

# 5. Predict 100 ngày tiếp theo (recursive)
last_close = df["close"].iloc[-1]   # giá close cuối cùng trong dữ liệu
future_preds = []
current_close = last_close

for i in range(100):
    next_close = model_full.predict([[current_close]])[0]
    future_preds.append(next_close)
    current_close = next_close

# 6. Tạo dataframe submission: id 1..100, close = dự báo
submission = pd.DataFrame({
    "id": np.arange(1, 101),
    "close": future_preds
})

submission.to_csv("submission.csv", index=False)

print("Saved submission.csv, preview:")
print(submission.head())

# 547.0701

MSE (train 80%, test 20%): 2.944831185160196
Model full data - Intercept: 0.07927395153159011
Model full data - Coef: [1.00013321]
Saved submission.csv, preview:
   id       close
0   1  122.015515
1   2  122.111043
2   3  122.206583
3   4  122.302136
4   5  122.397702
